# RAG pomocou služby Azure AI Agent a Semantic Kernel

Tento úryvok kódu ukazuje, ako vytvoriť a spravovať Azure AI agenta pre generovanie s podporou vyhľadávania (RAG) pomocou `Azure AI Agent Service` a `Semantic Kernel`. Agent spracováva používateľské otázky na základe získaného kontextu a poskytuje presné odpovede.


Oprava verzie SQLite  
Ak narazíte na chybu:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Odkomentujte tento blok kódu na začiatku vášho notebooku:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importovanie balíkov
Nasledujúci kód importuje potrebné balíky:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Generovanie s podporou vyhľadávania pomocou Semantic Kernel a Azure AI Agent Service

Táto ukážka demonštruje, ako použiť **Azure AI Agent Service** na vykonávanie **generovania s podporou vyhľadávania (RAG)** kombináciou jazykového modelu s kontextom špecifickým pre danú oblasť z nahraného dokumentu.

### Ako to funguje

1. **Nahranie dokumentu**: Markdown súbor (document.md) obsahujúci informácie (poistná politika cestovného poistenia spoločnosti Contoso) je nahraný do agentovej služby.

2. **Vytvorenie vektorového úložiska**: Dokument je indexovaný do vektorového úložiska, aby sa umožnilo sémantické vyhľadávanie jeho obsahu.

3. **Konfigurácia agenta**: Agent je inicializovaný pomocou modelu `gpt-4o` s nasledujúcimi prísnymi pokynmi:
   - Odpovedať iba na otázky na základe získaného obsahu z dokumentu.
   - Odmietnuť odpovedať, ak je otázka mimo rozsahu.

4. **Integrácia nástroja na vyhľadávanie v súboroch**: `FileSearchTool` je zaregistrovaný u agenta, čo umožňuje modelu počas inferencie vyhľadávať a získavať relevantné úryvky z indexovaného dokumentu.

5. **Interakcia s používateľom**: Používatelia môžu klásť otázky. Ak sa v dokumente nájdu relevantné informácie, agent vygeneruje odpoveď založenú na faktoch.  
   Ak nie, agent výslovne odpovie, že dokument neobsahuje dostatočné informácie.


### Hlavná funkcia



Uistite sa, že najskôr spustíte `az login` pomocou Azure CLI, aby bol poskytnutý správny autentifikačný kontext pri používaní `DefaultAzureCredential`. Služba Azure AI Agent Service nepoužíva API kľúče.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

# Úvod

Tento dokument poskytuje prehľad o tom, ako používať @@INLINE_CODE_0@@ na dosiahnutie optimálnych výsledkov. 

## Predpoklady

Pred začatím sa uistite, že máte nainštalované nasledujúce:

- @@INLINE_CODE_1@@
- @@INLINE_CODE_2@@

## Inštalácia

Ak chcete nainštalovať @@INLINE_CODE_3@@, postupujte podľa týchto krokov:

1. Otvorte terminál.
2. Spustite príkaz:

   ```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```

3. Overte inštaláciu pomocou:

   @@CODE_BLOCK_1@@

## Použitie

### Základné použitie

Na spustenie základnej operácie použite:

@@CODE_BLOCK_2@@

### Pokročilé možnosti

Pre pokročilé možnosti môžete pridať nasledujúce parametre:

@@CODE_BLOCK_3@@

[!TIP] Pre najlepšie výsledky sa odporúča používať najnovšiu verziu @@INLINE_CODE_4@@.

## Riešenie problémov

Ak narazíte na chyby, skúste:

1. Skontrolovať, či sú všetky závislosti správne nainštalované.
2. Spustiť diagnostický príkaz:

   @@CODE_BLOCK_4@@

3. Pozrieť si dokumentáciu na @@INLINE_CODE_5@@.

[!NOTE] Ak problém pretrváva, kontaktujte podporu na @@INLINE_CODE_6@@.

## Záver

Dodržiavaním vyššie uvedených krokov by ste mali byť schopní úspešne používať @@INLINE_CODE_7@@. Pre ďalšie informácie navštívte @@INLINE_CODE_8@@.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
